# Library

In [2]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta
import quandl
from datetime import datetime
from openpyxl import load_workbook

# Data Set

In [3]:
# GitHub raw file URL
xlsx_url = "https://raw.githubusercontent.com/elcacique69/DCF---Portfolio-Acquisition-Tool/main/Data_Set_Closing.xlsx"

df = pd.read_excel(xlsx_url, sheet_name="Planned Portfolio")
df_asset_register = pd.read_excel(xlsx_url, sheet_name="Updated Asset Register")
df_debt = pd.read_excel(xlsx_url, sheet_name="Debt")

In [ ]:
def bank_convenants():
    pass

# Drawdown Amount 

In [6]:
# Calculate the Portfolio Purchase Price
purchase_price = df['Purchase Price'].sum()

# Minimal amount for drawdown
minimal_amount = 3000000

# Outstanding Facility Amount
debt = df_debt['Drawdown'].sum()
facility = 35000000
outstanding_facility = facility - debt

# If statement for Purchase Amount Covenant
if purchase_price > minimal_amount:
    if purchase_price <= outstanding_facility:
        print("The Drawdown minimal amount is respected")
    else:
        print("BREACH: The purchase amount exceeds the facility capacity.")
else:
    print("BREACH: minimal amount for drawdown is 3,000,000.00 USD")

The Drawdown minimal amount is respected


# Manufacturer Covenants

In [19]:
manufacturer_list = ["FUWA", "Singamas", "Maersk", "Dong Fang", "CXI", "Seabox",
                     "China Shipping Container Lines (CSCL)", "Textainer Group Holdings Limited",
                     "COSCO Shipping Development", "Hoover Ferguson Group"]

df_not_manuf = df[~df['Manufacturer'].isin(manufacturer_list)]




,Serial Number,Type,Unit,CEU,Manufacturer,Manufacturing Date,Vintage,Current Status,Lessee,Contract Number,Contract Type,End Contract date,Per Diem (Unit),Purchase Price,Closing Date
4402,TGCU2339230,20'DC,1,1.0,CIMC,2023-03-17,2023,Off Lease,ONE,4439,Long-term Contract,2035-02-28,0.74,2191.0,2023-06-12
4403,TGCU2339245,20'DC,1,1.0,CIMC,2023-03-17,2023,Off Lease,ONE,4439,Long-term Contract,NaT,0.74,2191.0,2023-06-12
4404,TGCU2339250,20'DC,1,1.0,CIMC,2023-03-17,2023,Off Lease,ONE,4439,Long-term Contract,NaT,0.74,2191.0,2023-06-12
4405,TGCU2339266,20'DC,1,1.0,CIMC,2023-03-17,2023,Off Lease,ONE,4439,Long-term Contract,NaT,0.74,2191.0,2023-06-12
4406,TGCU2339271,20'DC,1,1.0,CIMC,2023-03-17,2023,Off Lease,ONE,4439,Long-term Contract,NaT,0.74,2191.0,2023-06-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4677,TGCU2341983,20'DC,1,1.0,CIMC,2023-03-17,2023,Off Lease,ONE,4439,Long-term Contract,NaT,0.74,2191.0,2023-06-12
4678,TGCU2341999,20'DC,1,1.0,CIMC,2023-03-17,2023,Off Lease,ONE,4439,Long-term Contract,NaT,0.74,2191.0,2023-06-12
4679,TGCU2342001,20'DC,1,1.0,CIMC,2023-03-17,2023,Off Lease,ONE,4439,Long-term Contract,NaT,0.74,2191.0,2023-06-12
4680,TGCU2342017,20'DC,1,1.0,CIMC,2023-03-17,2023,Off Lease,ONE,4439,Long-term Contract,NaT,0.74,2191.0,2023-06-12


In [40]:
# List of manufacturers
manufacturer_list = ["CIMC", "Singamas", "Maersk", "Dong Fang", "CXI", "Seabox",
                     "China Shipping Container Lines (CSCL)", "Textainer Group Holdings Limited",
                     "COSCO Shipping Development", "Hoover Ferguson Group"]

# Check if the manufacturer is in the list or not
def check_manufacturer(manufacturer):
    if manufacturer in manufacturer_list:
        return "OK"
    else:
        return "Not in the list"

    # Iterate through the dataframe and check the manufacturers
    results = []
    for index, row in df.iterrows():
        manufacturer = row['Manufacturer']
        if manufacturer in manufacturer_list:
            results.append("OK")
        else:
            results.append(row)

    # If there are any non-matching manufacturers, create a dataframe
    if any(isinstance(x, pd.Series) for x in results):
        non_matching_df = pd.concat(results, axis=1).T
    else:
        non_matching_df = None

    # Export non-matching containers to Excel
    if non_matching_df is not None:
        export_path = "/Users/carlosjosegonzalezacevedo/Documents/NEOMA/Thesis/DCF Container portfolio acquisition model/DCF---Portfolio-Acquisition-Tool/containers_wrong_manufacturer.xlsx"
        sheet_name = "Wrong Manufacturer List"
        non_matching_df.to_excel(export_path, index=False, sheet_name=sheet_name)
        print(f"Non-matching containers exported to: {export_path} (Sheet: {sheet_name})")
    else:
        print("All containers have matching manufacturers.")

Non-matching containers exported to: /Users/carlosjosegonzalezacevedo/Documents/NEOMA/Thesis/DCF Container portfolio acquisition model/DCF---Portfolio-Acquisition-Tool/containers_wrong_manufacturer.xlsx (Sheet: Wrong Manufacturer List)


# Concentration Covenants

In [15]:
# Define NBV
NBV = df_asset_register['NBV'].sum()

# Define the list of customers
lessees = ['Shipping Line 1', 'Shipping Line 2', 'Shipping Line 3', 'Shipping Line 4', 'Shipping Line 5', 'Shipping Line 6', 'Shipping Line 7', 'Shipping Line 8']

# Create a list to store the NBV sums for each customer
lessee_nbv_sums = []

# Flag to keep track of concentration breaches
concentration_breaches = False

# Iterate over each customer
for lessee in lessees:
    # Filter the DataFrame for rows where 'Lessee' is the current customer
    lessee_df = df_asset_register[df_asset_register['Lessee'] == lessee]

    # Calculate the sum of 'NBV' for the current customer
    nbv_sum = lessee_df['NBV'].sum() / NBV * 100
    lessee_nbv_sums.append(nbv_sum)

    # Print the sum of 'NBV' for the current customer
    print(f"NBV concentration: {lessee}: {nbv_sum:,.2f}%")

    # Check if the NBV concentration exceeds the threshold
    if nbv_sum > 30:
        concentration_breaches = True
        print("BREACH: NBV concentration above 30%!")

    # Check if ZIM NBV concentration exceeds the threshold
    if lessee == 'ZIM' and nbv_sum > 15:
        concentration_breaches = True
        print("BREACH: ZIM NBV concentration above 15%!")

# Check if there were any concentration breaches
if not concentration_breaches:
    print("No concentration breaches have been observed.")

NBV concentration: Shipping Line 1: 22.91%
NBV concentration: Shipping Line 2: 28.96%
NBV concentration: Shipping Line 3: 8.23%
NBV concentration: Shipping Line 4: 0.02%
NBV concentration: Shipping Line 5: 5.95%
NBV concentration: Shipping Line 6: 0.02%
NBV concentration: Shipping Line 7: 0.03%
NBV concentration: Shipping Line 8: 5.76%
No concentration breaches have been observed.


# Advance Rate

In [25]:
closing_date = datetime(2022, 5, 19)
current_date = datetime(2023, 5, 19)


def calculate_advance_rate(closing_date: datetime, current_date: datetime):
    # Termination date and first repayment date
    termination_date = closing_date + timedelta(days=30*27) #approximating months by 30 days
    first_repayment_date = closing_date + timedelta(days=30*12) # aprox months and 1 year after the closing date
    
    # Difference in months
    months_delta = (current_date.year - closing_date.year) * 12 + current_date.month - closing_date.month
    
    # Calculating the advance rate based on dates conditions
    if current_date <= closing_date:
        return "Invalid date. The current date should be after the closing date"
    elif current_date <= first_repayment_date:
        return "The limit advance rate is: 66%"
    elif current_date <= termination_date:
        # Reduction of 1% for every 3 months after the first repayment date 
        advance_rate_reduction = ((months_delta - 12) // 3) * 1
        return max(65 - advance_rate_reduction, 61) # return the calculated advance rate as a numeric value
    else:
        return 61 # return the default advance rate as a numeric value

# Updated debt
updated_debt = df['Purchase Price'].sum() + df_debt['Total'].sum()
    
# Updated Asset Register
updated_nbv = df_asset_register['NBV'].sum()

# Advance Rate
updated_advance_rate = (updated_debt / updated_nbv) * 100

if updated_advance_rate > calculate_advance_rate(closing_date, current_date):
    print("BREACH: The updated advance rate exceeds the calculated advance rate.")

print(f'Updated Advance Rate: {updated_advance_rate:.2f}%')
print(f'The updated Debt: {updated_debt:,.2f} USD')
print(f'The updated NBV: {updated_nbv:,.2f} USD')

Updated Advance Rate: 63.68%
The updated Debt: 32,498,506.87 USD
The updated NBV: 51,033,201.71 USD


# Age Covenant

In [39]:
closing_date = datetime(2023, 6, 12)  # Set the closing date

# Convert the "Manufacturing Date" column to datetime if it's not already in datetime format
df['Manufacturing Date'] = pd.to_datetime(df['Manufacturing Date'])

# Calculate the age for each container row
df['Age'] = (closing_date - df['Manufacturing Date']).dt.days

# Calculate the weighted age using the "Age" and "Purchase Price" columns
df['Weighted Age'] = df['Age'] * df['Purchase Price']

# Calculate the weighted average age
weighted_average_age = df['Weighted Age'].sum() / df['Purchase Price'].sum() / 365

# Print the weighted average age
print(f"Weighted NBV Average Age: {weighted_average_age:.2f} years")

# Check if the weighted average age is above 9 and print a message
if weighted_average_age > 9:
    print("BREACH: The weighted average age is above 9 years.")

Weighted NBV Average Age: 7.53 years


# NBV by CEU

In [49]:
updated_ceu = df_asset_register['CEU'].sum()

ceu_purchase_price = updated_nbv / updated_ceu

if ceu_purchase_price > 2900:
    print("BREACH: in contract the CEU price must be below 2900 USD")
    
print(f"Total CEU: {ceu:,.2f}")
print(f"CEU Purchase Price:{ceu_purchase_price:,.2f} USD")

Total CEU: 31,055.90
CEU Purchase Price:1,643.27 USD


# Average Remaining Lease Term

In [57]:
import pandas as pd
from datetime import datetime

# Filter containers manufactured after 2019
filtered_df = df[df['Vintage'] > 2019].copy()

# Calculate remaining lease term
closing_date = datetime.now()  # Assuming the closing_date is the current date
filtered_df['Remaining Lease Term'] = (filtered_df['End contract date'] - closing_date).dt.days

# Calculate weighted average remaining lease term
weighted_average = (filtered_df['Remaining Lease Term'] * filtered_df['Purchase Price']).sum() / filtered_df['Purchase Price'].sum()

if weighted_average < 5:
    print("BREACH: the minimum weighted remaining lease term for equipment manufactured after 2019 must be 5 years")

print("Average Remaining Lease Term Weighted with Purchase Price for Containers Manufactured after 2019:", weighted_average)


Average Remaining Lease Term Weighted with Purchase Price for Containers Manufactured after 2019: 15.275


# Finance Lease

In [67]:
finance_lease_df = df_asset_register[df_asset_register['Lease Type'] == "Finance Lease"]
finance_lease_nbv = finance_lease_df['NBV'].sum()
finance_lease_proportion = finance_lease_nbv / NBV * 100

if finance_lease_proportion > 30:
    print("BREACH: The Finance Lease proportion needs to be below 30%")

print(f"Finance lease proportion: {finance_lease_proportion:,.2f}%")

Finance lease proportion: 1.79%
